In [17]:
import pandas as pd

df = pd.read_csv("../data/Disease precaution.csv")

print(df.shape)
display(df.head())


(41, 5)


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [18]:
# View column names
print(df.columns)

# Check for missing values
print(df.isnull().sum())


Index(['Disease', 'Precaution_1', 'Precaution_2', 'Precaution_3',
       'Precaution_4'],
      dtype='object')
Disease         0
Precaution_1    0
Precaution_2    0
Precaution_3    1
Precaution_4    1
dtype: int64


In [19]:
symptom_df = pd.read_csv("../data/disease_symptom.csv")

print(symptom_df.shape)
display(symptom_df.head())


(4920, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Fill NaN values with empty strings
symptom_df_filled = symptom_df.fillna("")

# Separate target
y = symptom_df_filled["Disease"]

# Combine all symptom columns into a single list per row
symptom_cols = [col for col in symptom_df_filled.columns if col.startswith("Symptom")]
X_symptoms = symptom_df_filled[symptom_cols]

# One-hot encode symptoms
X = pd.get_dummies(X_symptoms.stack()).groupby(level=0).sum()

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)


Feature matrix shape: (4920, 132)
Target shape: (4920,)


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize model
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", round(accuracy, 4))


Test Accuracy: 1.0


In [22]:
import numpy as np

# Get prediction probabilities
y_proba = model.predict_proba(X_test)

# Get class labels
classes = model.classes_

def get_top_k_predictions(proba_row, classes, k=3):
    top_k_idx = np.argsort(proba_row)[-k:][::-1]
    return [(classes[i], proba_row[i]) for i in top_k_idx]

# Example: Top-3 predictions for the first test sample
sample_idx = 0
top_3 = get_top_k_predictions(y_proba[sample_idx], classes, k=3)

print("Top-3 predicted diseases:")
for disease, score in top_3:
    print(f"{disease} — confidence: {score:.2f}")


Top-3 predicted diseases:
Hypertension  — confidence: 1.00
hepatitis A — confidence: 0.00
Common Cold — confidence: 0.00


In [23]:
# Get symptom list for the selected sample
sample_symptoms = X_test.iloc[sample_idx]
active_symptoms = sample_symptoms[sample_symptoms == 1].index.tolist()

print("User-reported symptoms:")
print(active_symptoms)


User-reported symptoms:
[' chest_pain', ' dizziness', ' lack_of_concentration', ' loss_of_balance']


In [24]:
def generate_explanation(symptoms, predictions):
    explanation = f"""
The following health conditions are suggested based on the reported symptoms.
This is a preliminary assessment and not a medical diagnosis.

Reported symptoms:
{', '.join(symptoms)}

Possible conditions and explanations:
"""
    for disease, confidence in predictions:
        explanation += f"""
- {disease} (confidence: {confidence:.2f}):
  This condition is suggested because it is commonly associated with symptoms such as
  {', '.join(symptoms[:2])}. However, symptoms may overlap with other conditions.

"""
    explanation += """
Please consult a qualified healthcare professional for an accurate diagnosis and treatment.
"""
    return explanation

# Generate explanation text
explanation_text = generate_explanation(active_symptoms, top_3)
print(explanation_text)



The following health conditions are suggested based on the reported symptoms.
This is a preliminary assessment and not a medical diagnosis.

Reported symptoms:
 chest_pain,  dizziness,  lack_of_concentration,  loss_of_balance

Possible conditions and explanations:

- Hypertension  (confidence: 1.00):
  This condition is suggested because it is commonly associated with symptoms such as
   chest_pain,  dizziness. However, symptoms may overlap with other conditions.


- hepatitis A (confidence: 0.00):
  This condition is suggested because it is commonly associated with symptoms such as
   chest_pain,  dizziness. However, symptoms may overlap with other conditions.


- Common Cold (confidence: 0.00):
  This condition is suggested because it is commonly associated with symptoms such as
   chest_pain,  dizziness. However, symptoms may overlap with other conditions.


Please consult a qualified healthcare professional for an accurate diagnosis and treatment.



In [25]:
# Load disease precautions
precaution_df = pd.read_csv("../data/Disease precaution.csv")

precaution_df.head()


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [26]:
def get_precautions(disease_name, precaution_df):
    row = precaution_df[precaution_df["Disease"] == disease_name]
    if row.empty:
        return []
    precautions = row.iloc[0, 1:].dropna().tolist()
    return precautions

# Example: show precautions for top predictions
for disease, _ in top_3:
    precautions = get_precautions(disease, precaution_df)
    print(f"\nPrecautions for {disease}:")
    for p in precautions:
        print("-", p)



Precautions for Hypertension :
- meditation
- salt baths
- reduce stress
- get proper sleep

Precautions for hepatitis A:
- Consult nearest hospital
- wash hands through
- avoid fatty spicy food
- medication

Precautions for Common Cold:
- drink vitamin c rich drinks
- take vapour
- avoid cold food
- keep fever in check


In [27]:
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [28]:
# Save training feature names
trained_features = X_train.columns


In [35]:
import pickle

# Save trained model
pickle.dump(model, open("model.pkl", "wb"))

# Save trained feature names
pickle.dump(trained_features, open("features.pkl", "wb"))

print("Model and features saved successfully!")


Model and features saved successfully!


In [ ]:
def predict_from_symptoms(input_symptoms, top_k=3):
    input_vector = pd.Series(0, index=trained_features)
    unknown_symptoms = []

    for symptom in input_symptoms:
        if symptom in input_vector.index:
            input_vector[symptom] = 1
        else:
            unknown_symptoms.append(symptom)

    input_df = pd.DataFrame([input_vector])
    proba = model.predict_proba(input_df)[0]
    classes = model.classes_

    top_idx = np.argsort(proba)[-top_k:][::-1]
    results = [(classes[i], proba[i]) for i in top_idx]

    return results, unknown_symptoms




In [30]:
user_input = input("Enter symptoms separated by commas: ")

manual_symptoms = [s.strip() for s in user_input.split(",")]

manual_top3 = predict_from_symptoms(
    manual_symptoms,
    model,
    trained_features,
    top_k=3
)

print("\nTop-3 predicted diseases:")
for disease, score in manual_top3:
    print(f"{disease} — confidence: {score:.2f}")



Top-3 predicted diseases:
Tuberculosis — confidence: 0.11
Dimorphic hemmorhoids(piles) — confidence: 0.06
Alcoholic hepatitis — confidence: 0.04


In [31]:
def get_precautions(disease_name, precaution_df):
    row = precaution_df[precaution_df["Disease"] == disease_name]
    if row.empty:
        return []
    return row.iloc[0, 1:].dropna().tolist()


In [32]:
def generate_explanation(symptoms, disease, confidence):
    return (
        f"The condition '{disease}' is suggested with a confidence of {confidence:.2f} "
        f"because it is commonly associated with symptoms such as "
        f"{', '.join(symptoms[:2])}. "
        f"This is a preliminary assessment and not a medical diagnosis."
    )


In [33]:
# ---- FULL MANUAL SYMPTOM PIPELINE ---- #

user_input = input("Enter symptoms separated by commas: ")
manual_symptoms = [s.strip() for s in user_input.split(",")]

manual_top3 = predict_from_symptoms(
    manual_symptoms,
    model,
    trained_features,
    top_k=3
)

print("\n🔍 Top-3 Predicted Diseases with Explanations & Precautions:\n")

for disease, score in manual_top3:
    print(f"🩺 Disease: {disease}")
    print(f"📊 Confidence: {score:.2f}")
    
    # Explanation
    explanation = generate_explanation(manual_symptoms, disease, score)
    print(f"💡 Explanation: {explanation}")
    
    # Precautions
    precautions = get_precautions(disease, precaution_df)
    if precautions:
        print("🛡️ Precautions:")
        for p in precautions:
            print(f"  - {p}")
    else:
        print("🛡️ Precautions: Not available")
    
    print("-" * 60)



🔍 Top-3 Predicted Diseases with Explanations & Precautions:

🩺 Disease: Tuberculosis
📊 Confidence: 0.11
💡 Explanation: The condition 'Tuberculosis' is suggested with a confidence of 0.11 because it is commonly associated with symptoms such as chest_pain, dizziness. This is a preliminary assessment and not a medical diagnosis.
🛡️ Precautions:
  - cover mouth
  - consult doctor
  - medication
  - rest
------------------------------------------------------------
🩺 Disease: Dimorphic hemmorhoids(piles)
📊 Confidence: 0.06
💡 Explanation: The condition 'Dimorphic hemmorhoids(piles)' is suggested with a confidence of 0.06 because it is commonly associated with symptoms such as chest_pain, dizziness. This is a preliminary assessment and not a medical diagnosis.
🛡️ Precautions:
  - avoid fatty spicy food
  - consume witch hazel
  - warm bath with epsom salt
  - consume alovera juice
------------------------------------------------------------
🩺 Disease: Alcoholic hepatitis
📊 Confidence: 0.04
💡 